# **Chapter 11: Training Deep Neural Networks**

## **1. Pendahuluan**

[cite_start]Di bab sebelumnya, kita telah melatih jaringan saraf tiruan sederhana yang cukup dangkal. Namun, bagaimana jika kita perlu menangani masalah yang sangat kompleks, seperti mendeteksi ratusan jenis objek dalam gambar resolusi tinggi?[cite: 1290]. Kita akan membutuhkan Deep Neural Network (DNN) yang jauh lebih dalam, mungkin dengan puluhan lapisan dan ratusan neuron per lapisan.

Melatih jaringan sedalam itu tidaklah mudah. Kita akan menghadapi berbagai tantangan:
* [cite_start]**Masalah Gradien:** Gradien bisa menghilang (*vanishing*) atau meledak (*exploding*) saat mengalir mundur melalui jaringan, membuat pelatihan lapisan bawah menjadi sangat sulit[cite: 1293, 1294, 1295].
* [cite_start]**Data Training:** Jaringan besar membutuhkan jumlah data yang sangat besar, yang mungkin sulit atau mahal untuk diberi label[cite: 1296].
* [cite_start]**Waktu Pelatihan:** Proses training bisa menjadi sangat lambat[cite: 1297].
* [cite_start]**Overfitting:** Dengan jutaan parameter, model berisiko tinggi menghafal data training alih-alih mempelajarinya[cite: 1298].

[cite_start]Dalam bab ini, kita akan membahas solusi untuk setiap masalah tersebut, mulai dari teknik inisialisasi bobot, penggunaan optimizer yang lebih cepat, hingga teknik regularisasi canggih[cite: 1299, 1300, 1301, 1302].

## **2. Masalah Vanishing & Exploding Gradients**

### **Konsep Dasar**
Algoritma Backpropagation bekerja dengan merambatkam gradien kesalahan dari output layer kembali ke input layer. [cite_start]Namun, dalam jaringan yang dalam, gradien ini sering kali menjadi semakin kecil saat mencapai lapisan bawah (*vanishing gradients*) atau sebaliknya, menjadi sangat besar (*exploding gradients*)[cite: 1309, 1310, 1311].

[cite_start]Penyebab utamanya sering kali adalah kombinasi dari fungsi aktivasi (seperti sigmoid) yang mengalami saturasi dan metode inisialisasi bobot yang kurang tepat[cite: 1316, 1317]. [cite_start]Seperti diilustrasikan pada Gambar 11-1 di buku, fungsi sigmoid memadamkan gradien ketika inputnya terlalu besar atau terlalu kecil[cite: 1320].

### **Solusi 1: Inisialisasi Glorot dan He**
[cite_start]Agar sinyal mengalir dengan baik, varians output dari setiap lapisan harus sama dengan varians inputnya[cite: 1346]. [cite_start]Xavier Glorot dan Yoshua Bengio mengusulkan strategi inisialisasi bobot (Glorot Initialization) yang terbukti bekerja sangat baik[cite: 1347, 1348].

[cite_start]Untuk fungsi aktivasi ReLU (dan variannya), strategi yang lebih cocok disebut **He Initialization**[cite: 1364].

### **Implementasi di Keras**
Secara default, Keras menggunakan inisialisasi Glorot. Kita bisa mengubahnya menjadi He Initialization dengan mudah.

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Menggunakan He Initialization dengan distribusi Normal untuk lapisan Dense
# Ini sangat disarankan jika menggunakan fungsi aktivasi ReLU atau variannya
layer = keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

print("Layer berhasil dibuat dengan He Initialization.")

**Interpretasi:**
Dengan menggunakan `kernel_initializer="he_normal"`, kita memastikan bobot awal jaringan diatur sedemikian rupa sehingga varians sinyal terjaga saat melewati lapisan ReLU, mencegah gradien menghilang di awal pelatihan.

### **Solusi 2: Fungsi Aktivasi Non-Saturating**
Masalah lain dari sigmoid adalah saturasi. Oleh karena itu, fungsi aktivasi modern lebih disukai:

* [cite_start]**Leaky ReLU:** Varian ReLU yang tidak pernah "mati" karena memiliki kemiringan kecil ($\alpha$) untuk nilai negatif[cite: 1388, 1389].
* [cite_start]**ELU (Exponential Linear Unit):** Mirip ReLU tapi lebih mulus dan bisa bernilai negatif, yang membantu rata-rata output mendekati 0[cite: 1419, 1446].
* [cite_start]**SELU (Scaled ELU):** Varian ELU yang memungkinkan jaringan melakukan *self-normalization* (menjaga mean 0 dan varians 1 di setiap layer) jika kondisi tertentu terpenuhi[cite: 1454].

[cite_start]Urutan prioritas umum untuk dicoba: SELU > ELU > Leaky ReLU > ReLU > Tanh > Sigmoid[cite: 1464].

In [ ]:
# Contoh penggunaan Leaky ReLU
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2), # Menambahkan layer aktivasi secara terpisah
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dense(10, activation="softmax")
])

# Contoh penggunaan SELU (perlu inisialisasi lecun_normal)
layer_selu = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")

### **Solusi 3: Batch Normalization**
Meskipun inisialisasi yang baik membantu di awal, distribusi data bisa berubah selama pelatihan. [cite_start]**Batch Normalization (BN)** mengatasi ini dengan menambahkan operasi di setiap layer yang memusatkan (zero-center) dan menormalisasi input, lalu menskalakan dan menggesernya[cite: 1488, 1489, 1490].

Ini memungkinkan model belajar skala dan mean optimal untuk setiap layer. [cite_start]BN bertindak seperti regularizer dan memungkinkan penggunaan learning rate yang jauh lebih besar[cite: 1533, 1529].

### **Implementasi BN di Keras**
Kita biasanya menambahkan BN setelah lapisan *Dense* atau *Convolutional*.

In [ ]:
# Implementasi Batch Normalization dalam model Sequential
model_bn = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(), # Opsional: menormalisasi input awal
    keras.layers.Dense(300, use_bias=False), # Bias tidak perlu karena BN punya parameter shift
    keras.layers.BatchNormalization(), # BN sebelum aktivasi
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

print("Model dengan Batch Normalization siap.")

**Interpretasi:**
Layer `BatchNormalization` memiliki parameter yang dapat dilatih ($\gamma, \beta$) dan yang tidak dapat dilatih (moving mean/variance). [cite_start]Meskipun menambah kompleksitas komputasi per epoch, BN sering kali membuat konvergensi jauh lebih cepat sehingga total waktu pelatihan berkurang[cite: 1541]. [cite_start]Perhatikan `use_bias=False` pada lapisan Dense sebelumnya karena BN sudah menangani bias (offset)[cite: 1614].

## **3. Transfer Learning**

Jangan melatih model dari nol jika tidak perlu! [cite_start]**Transfer Learning** adalah teknik menggunakan kembali lapisan bawah dari model yang sudah dilatih pada tugas serupa[cite: 1669, 1670]. [cite_start]Ini mempercepat pelatihan secara drastis dan mengurangi kebutuhan data[cite: 1673].

[cite_start]Seperti pada Gambar 11-4 di buku, kita biasanya membekukan (*freeze*) lapisan bawah (karena fitur level rendah seperti tepi/garis bersifat universal) dan hanya melatih ulang lapisan atas[cite: 1704].

In [ ]:
# Anggap model_A adalah model yang sudah dilatih sebelumnya
model_A = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])

# Kita ingin menggunakan kembali layer model_A kecuali output layer
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid")) # Task baru binary classification

# PENTING: Bekukan layer yang digunakan kembali agar bobotnya tidak rusak di awal
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

# Compile model setelah membekukan layer
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

print("Model B siap untuk transfer learning (layer awal dibekukan).")

## **4. Optimizer yang Lebih Cepat**

Gradient Descent standar sering kali terlalu lambat. Kita bisa menggunakan optimizer yang lebih canggih:

* **Momentum Optimization:** Mempercepat di turunan curam, seperti bola menggelinding. [cite_start]Menggunakan momentum dari gradien sebelumnya[cite: 1815, 1823].
* [cite_start]**Nesterov Accelerated Gradient (NAG):** Mengukur gradien sedikit di depan posisi saat ini, lebih akurat dan cepat dari momentum biasa[cite: 1849, 1856].
* [cite_start]**RMSProp:** Beradaptasi dengan kemiringan yang berbeda di setiap dimensi, mencegah osilasi berlebih[cite: 1906, 1910].
* **Adam:** Menggabungkan Momentum dan RMSProp. [cite_start]Ini adalah pilihan default yang sangat populer[cite: 1923].

### **Implementasi Optimizer**
Mengganti optimizer di Keras sangat mudah.

In [ ]:
# Menggunakan SGD dengan Momentum
optimizer_sgd = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

# Menggunakan Nesterov
optimizer_nesterov = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

# Menggunakan RMSProp
optimizer_rmsprop = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

# Menggunakan Adam (Seringkali pilihan terbaik)
optimizer_adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

print("Berbagai optimizer telah diinisialisasi.")

## **5. Learning Rate Scheduling**

Menemukan learning rate yang tepat sangat krusial. [cite_start]Alih-alih menggunakan nilai konstan, strategi **Learning Rate Scheduling** memulai dengan learning rate besar dan mengecilkannya seiring waktu[cite: 2000].

[cite_start]Salah satu cara efektif adalah **Exponential Scheduling**, di mana learning rate turun sebesar faktor 10 setiap $s$ langkah[cite: 2011].

In [ ]:
# Implementasi Exponential Decay di Keras
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)

optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
print("Optimizer dengan learning rate scheduling siap digunakan.")

## **6. Regularisasi: Menghindari Overfitting**

Untuk DNN dengan jutaan parameter, regularisasi wajib dilakukan. Selain L1/L2 regularization, teknik yang paling populer adalah **Dropout**.

### **Dropout**
[cite_start]Pada setiap langkah pelatihan, setiap neuron (kecuali output) memiliki probabilitas $p$ untuk diabaikan sementara[cite: 2125]. [cite_start]Ini memaksa neuron untuk mandiri dan tidak bergantung pada neuron tetangganya, membuat jaringan lebih *robust*[cite: 2145].

### **MC Dropout**
Teknik menarik lainnya adalah Monte Carlo (MC) Dropout, yang membiarkan dropout tetap aktif saat prediksi (testing). [cite_start]Dengan melakukan prediksi berulang kali dan merata-ratakannya, kita mendapatkan estimasi ketidakpastian model yang lebih baik (Bayesian approximation)[cite: 2191, 2208].

In [ ]:
# Implementasi Dropout
model_dropout = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), # Drop 20% input neurons
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2), # Drop 20% hidden neurons
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

# Implementasi MC Dropout (tanpa perlu training ulang, hanya saat inferensi)
import numpy as np

# Fungsi untuk melakukan prediksi MC Dropout
def predict_mc_dropout(model, X, n_samples=100):
    # training=True mengaktifkan dropout layer saat predict
    y_probas = np.stack([model(X, training=True) for sample in range(n_samples)])
    return y_probas.mean(axis=0) # Rata-rata dari 100 prediksi

print("Model Dropout dan fungsi MC Dropout siap.")

**Interpretasi Hasil:**
Dropout bertindak seperti melatih ribuan jaringan saraf kecil yang berbeda dan merata-ratakannya (ensemble). [cite_start]MC Dropout meningkatkan akurasi dan memberikan estimasi *confidence* yang lebih realistis daripada softmax biasa, yang sering kali terlalu percaya diri[cite: 2245, 2251].

## **7. Kesimpulan**

Dalam bab ini, kita telah mempelajari berbagai teknik untuk melatih DNN yang dalam dan kompleks:
1.  **Inisialisasi He/Glorot** dan fungsi aktivasi **ELU/SELU** mengatasi masalah vanishing gradients.
2.  **Batch Normalization** menstabilkan pelatihan dan mempercepat konvergensi.
3.  **Transfer Learning** memungkinkan kita memanfaatkan model yang sudah ada untuk tugas baru dengan data sedikit.
4.  **Optimizer canggih** (seperti Adam) dan **Learning Rate Scheduling** mempercepat proses pencarian solusi optimal.
5.  **Dropout** adalah regularisasi ampuh untuk mencegah overfitting.

Dengan "kotak perkakas" ini, Anda sekarang siap untuk membangun dan melatih model Deep Learning skala besar yang mampu menyelesaikan masalah dunia nyata yang kompleks.